In [40]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import KFold
from intra import IntrA
from models.pointnet import PointNetCls
from utils import train_model, eval_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
dataset = IntrA("~/Documents/Datasets/IntrA", dataset="generated", npoints=1024, exclude_seg=True)

tacc = 0
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for fold, (train_ids, test_ids) in enumerate(kf.split(dataset), start=1):
    model = PointNetCls(k=2, feature_transform=False)
    # if opt.model != "":
    #     model.load_state_dict(torch.load(opt.model))

    train_srs, test_srs = SubsetRandomSampler(train_ids), SubsetRandomSampler(test_ids)
    train_dl = DataLoader(dataset, batch_size=8, sampler=train_srs)
    test_dl = DataLoader(dataset, batch_size=8, sampler=test_srs)
    train_model(model, train_dl, test_dl)
    facc = eval_model(model, test_dl)
    tacc += facc
    print(f"Fold: {fold}. Fold acc: {facc}. Overall acc: {tacc/fold}.")

KeyboardInterrupt: 

In [50]:
dataset = IntrA("~/Documents/Datasets/IntrA", dataset="generated", npoints=1024, exclude_seg=True)
trn, tst = torch.utils.data.random_split(dataset, [0.8, 0.2], torch.Generator().manual_seed(42))

model = PointNetCls(k=2, feature_transform=False)
train_dl = DataLoader(trn, batch_size=8)
test_dl = DataLoader(tst, batch_size=8)
train_model(model, train_dl, test_dl, epochs=10)

/Users/jventers/miniconda3/envs/uni-dl/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [51]:
eval_model(model, test_dl)

100%|██████████| 48/48 [00:01<00:00, 24.86it/s]

vessel: 315/345=0.9130434782608695
aneurysm: 16/36=0.4444444444444444


0.868766404199475

In [ ]:
# torch.save(model.state_dict(), "%s/cls_model_%d.pth" % (opt.outf, epoch))